In [1]:
import requests

In [2]:
host = '172.20.233.208'

In [3]:
assert requests.get('http://%s:8079/health'%host).status_code == 200

In [5]:
resp = requests.post('http://%s:8079/verify'%host, '{"signedCredentials":{"issuer":"a"}}')
print(resp.status_code)
assert resp.status_code == 400 
assert resp.content

400


In [6]:
import json
creds = json.loads('{"@context":["https://www.w3.org/2018/credentials/v1",{"name":"schema:name"}],"type":["VerifiableCredential"],"credentialSubject":{"type":"Person","name":"Tejash"},"issuanceDate":"2021-08-27T10:57:57.237Z","issuer":"did:authorizedIssuer:23423#21","proof":{"type":"Ed25519Signature2018","created":"2022-07-19T05:11:21Z","verificationMethod":"did:authorizedIssuer:23423#21","proofPurpose":"assertionMethod","jws":"d455929f-8a6c-48e3-acac-633704d8c4a3..O6u5oAOiOeOt4nCHlJRRn04EyOoi7sRZHcJtvRXQqvKxQUfKdYjB9GJunO5pjnErWJC4U7gTctmUq34qrx98Bw"}}')
resp = requests.post('http://%s:4324/verify'%host, json={"signedCredentials":creds})
print(resp.status_code, resp.content)
assert resp.status_code==200
assert resp.json()["verified"]
resp.content
print("PASS")

ConnectionError: HTTPConnectionPool(host='172.20.233.208', port=4324): Max retries exceeded with url: /verify (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001605C5FEDA0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [18]:
resp = requests.post('http://%s:8079/sign'%host, json={"signedCredentials":""})
assert resp.status_code == 400

In [19]:
template="""{
        "@context": [
            "https://www.w3.org/2018/credentials/v1",
            {
                "@context": {
                    "@version": 1.1,
                    "@protected": true,
                    "SkillCertificate": {
                        "@id": "https://github.com/sunbird-specs/vc-specs#SkillCertificate",
                        "@context": {
                            "id": "@id",
                            "@version": 1.1,
                            "@protected": true,
                            "skills": {
                                "@id": "https://github.com/sunbird-specs/vc-specs#skills",
                                "@container": "@list"
                            },
                            "name":"schema:Text",
                            "refId":"schema:Text"
                        }
                    },
                    
                    "trainedOn": {
                        "@id": "https://github.com/sunbird-specs/vc-specs#trainedOn",
                        "@context": {
                            "name": "schema:Text"
                        }
                    }
                }
            }
        ],
        "type": [
            "VerifiableCredential","SkillCertificate"
        ],
        "credentialSubject": {
            "type": "SkillCertificate",
            "refId": "{{id}}",
            "name": "{{name}}",
            "skills": [{{#each skills}}"{{skill}}"{{#unless @last}},{{/unless}}{{/each}}]
        },
        "issuanceDate": "{{date}}",
        "issuer": "did:authorizedIssuer:23423#21",
        "evidence": [
            {
                "type": [
                    "Vaccination"
                ],
                "id": "https://sunbird.org/id"
            }
        ]
    }"""

In [20]:
entity = {
        "name": "a",
        "id": "123",
        "skills": [{"skill":"a"},{"skill":"b"}],
        "date":"2022-07-01T12:12:00Z"
    }

In [21]:
resp = requests.post('http://%s:4324/sign'%host, json={"data":entity, "credentialTemplate":template})
print('status', resp.status_code)
print(resp.content)
assert resp.status_code == 200
resp.json()['proof']['type'] == 'Ed25519Signature2018'
assert resp.json()["@context"][0] == 'https://www.w3.org/2018/credentials/v1'
signedCred = resp.json()

ConnectionError: HTTPConnectionPool(host='172.31.225.44', port=4324): Max retries exceeded with url: /sign (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001BE792CDFF0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [22]:
resp = requests.post('http://%s:4324/verify'%host, json={"signedCredentials":signedCred})
print(resp.status_code, resp.content)
assert resp.status_code == 200 
assert resp.json()["verified"]

NameError: name 'signedCred' is not defined